### Preparation 1: Install **Oracle Instant Client** software and **cx_Oracle** package

In [ ]:
# -- Install and setup Oracle Instant Client --
! wget -O oracleinstantclient.zip https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
! unzip -oq oracleinstantclient.zip
! mkdir -p /opt/oracle
! mv instantclient_19_6 /opt/oracle/
! sh -c "echo /opt/oracle/instantclient_19_6 > /etc/ld.so.conf.d/oracle-instantclient.conf"
! ln -s /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.0.so
! ldconfig


# -- Include cx_Oracle package --
! pip install cx_Oracle

--2022-05-03 02:06:02--  https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
Resolving download.oracle.com (download.oracle.com)... 23.192.208.88
Connecting to download.oracle.com (download.oracle.com)|23.192.208.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75462547 (72M) [application/zip]
Saving to: ‘oracleinstantclient.zip’

oracleinstantclient 100%[===================>]  71.97M  65.5MB/s    in 1.1s    

2022-05-03 02:06:04 (65.5 MB/s) - ‘oracleinstantclient.zip’ saved [75462547/75462547]

mv: cannot move 'instantclient_19_6' to '/opt/oracle/instantclient_19_6': Directory not empty
ln: failed to create symbolic link '/usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.0.so': File exists


### Preparation 2: Import **cx_Oracle** package and **connect** to Oracle Database

In [ ]:
# Import cx_Oracle package and connect to Oracle Database
import cx_Oracle

HOST_NAME = "imz409.ust.hk"
PORT_NUMBER = "1521"
SERVICE_NAME = "imz409"
USERNAME = "T403"
PASSWORD = "VM912"

dsn_tns = cx_Oracle.makedsn(HOST_NAME, PORT_NUMBER, service_name=SERVICE_NAME) 
conn = cx_Oracle.connect(user=USERNAME, password=PASSWORD, dsn=dsn_tns) 

c = conn.cursor()

### Preparation 3: Import **libraries**



In [ ]:
from google.colab.widgets import TabBar
import ipywidgets as widgets
import time
import pandas as pd
from ipywidgets import interact
from ipywidgets import Text
from ipywidgets import DatePicker
from ipywidgets import IntText
from ipywidgets import FloatText
from ipywidgets import Label
from ipywidgets import Layout
from datetime import date, datetime, timedelta
from ipywidgets import BoundedIntText
import re

### System

In [ ]:
tb_all = TabBar([
                 #Staff
                 'Staff Login', 'Staff Menu', 'Housekeeping Menu', 'Manager Dashboard', 'View Booking', 'Search Booking',
                 'Insert New Room Type', 'Insert New Room Info', 'Update Room Type Info', 'Update Room Availability',
                 'Checkin', 'Checkout', 'Reset Member Password', 'Input Discount Codes',
                 # Customer, Member
                 'Introduction', 'Member Login', 'Member Menu',
                 'Register Member', 'Room Searching', 'Make Booking', 'Redeem Reward'])

# a simple function for jumping between tabs
def showPage(tabName):
  with tb_all.output_to(tabName):
    display()

# defining global buttons
btnSLogout = widgets.Button(description="Logout")
def on_btnSLogout_clicked(b):
    stflogin_id_txt.value = ""
    stflogin_pwd_txt.value = ""
    stflogin_error.value = ""
    s3_2_update_label2.value = ""
    showPage("Staff Login")
btnFSBack = widgets.Button(description = "Back")
def on_btnFSBack_clicked(b):
  on_UPD_clear_button_clicked(b)
  s3_2_update_label1.value = ""
  INSs1_2_msg.value = ""
  on_CHK_clear_button_clicked(b)
  on_RST_clear_button_clicked(b)
  on_DIS_clear_button_clicked(b)
  on_INS_clear_button_clicked(b)
  showPage("Staff Menu")
btnBack = widgets.Button(description = "Main Menu")
def on_btnBack_clicked(b):
  on_MEM_clear_button_clicked(b)
  showPage("Introduction")
btnMBack = widgets.Button(description = "Back")
def on_btnMBack_clicked(b):
  showPage("Member Menu")

# create layout
page_layout = Layout(border='2px solid white', width='1100px', height='620px', margin='1% 5% 0% 5%', padding='1.5% 3% 1% 3%')

# global values/labels
spacer = Label("")
login_id = '0'
login_status = '0'
mem_noofstar=0

##########################################################
################ Introduction Tab - Begin ################ -- Allie
##########################################################
with tb_all.output_to('Introduction'):

  # create welcoming labels
  intro_label1 = Label("Welcome to")
  intro_label2 = Label("PetMyPet Hotel")
  intro_label3 = Label("Online Hotel Management System! :)")
  intro_label4 = Label("Powered by ISOM 3260 Team 403 ^__^")

  # create 3 buttons
  btnC1 = widgets.Button(description="Search & View Room")
  btnC2 = widgets.Button(description="Make Booking")
  btnC4 = widgets.Button(description="Register as Member")
  btnM = widgets.Button(description="Member Login")
  btnS = widgets.Button(description="Staff Login")

  # define the event handlers for the three buttons when they are being clicked
  def on_btnC1_button_clicked(b):
    rm.options = [row[0] for row in c.execute("SELECT RM_TYPENAME FROM ROOM_TYPE")]
    showPage("Room Searching")
  btnC1.on_click(on_btnC1_button_clicked)

  def on_btnC2_button_clicked(b):
    bk_rmtype_dropdown.options = [row[0] for row in c.execute("SELECT RM_TYPENAME FROM ROOM_TYPE")]
    showPage("Make Booking")
  btnC2.on_click(on_btnC2_button_clicked)

  def on_btnC4_button_clicked(b):
    showPage("Register Member")
  btnC4.on_click(on_btnC4_button_clicked)

  def on_btnS_button_clicked(b):
    stflogin_id_txt.value = ""
    stflogin_pwd_txt.value = ""
    stflogin_error.value = ""
    showPage("Staff Login")
  btnS.on_click(on_btnS_button_clicked)

  def on_btnM_button_clicked(b):
    phone_textbox.value = ""
    pwd_textbox.value = ""
    error_label.value = ""
    showPage("Member Login")
  btnM.on_click(on_btnM_button_clicked)

  # display the objects
  intro_row1HBox = widgets.HBox([intro_label1])
  intro_row2HBox = widgets.HBox([intro_label2])
  intro_row3HBox = widgets.HBox([intro_label3])
  intro_row4HBox = widgets.HBox([spacer])
  intro_row5HBox = widgets.HBox([btnC1])
  intro_row6HBox = widgets.HBox([btnC2])
  intro_row7HBox = widgets.HBox([btnC4])
  intro_row8HBox = widgets.HBox([spacer])
  intro_row9HBox = widgets.HBox([btnM])
  intro_row10HBox = widgets.HBox([btnS])
  intro_row11HBox = widgets.HBox([spacer])
  intro_row12HBox = widgets.HBox([intro_label4])

  display(widgets.VBox([intro_row1HBox, intro_row2HBox, intro_row3HBox, intro_row4HBox, intro_row5HBox, intro_row6HBox, 
                        intro_row7HBox, intro_row8HBox, intro_row9HBox, intro_row10HBox, intro_row11HBox, intro_row12HBox], layout=page_layout))
################ Introduction Tab - End ##################

##########################################################
############# c1 - Room Searching Tab - Begin ############ -- Gene
##########################################################
with tb_all.output_to('Room Searching'):

  ## Combo Box (Select room type)
  pickRoom = Label("Select a room type")
  rm_options = []
  c.execute("SELECT RM_TYPENAME FROM ROOM_TYPE")
  for row in c:
    rm_options.append(row[0])

  rm = widgets.Dropdown(options=rm_options, value=None)
  btnPick = widgets.Button(description="Pick")
  btnGoToBook = widgets.Button(description="Make a Booking!")
  btnC1Back = widgets.Button(description="Back")

  spacer1 = Label("---------------------------------------------------------------------------------------------------------------------------------")


  showInfo = Label("Selected room's information")

  style = {'description_width': '170px'}
  layout = {'width': '500px'}

  roomType_textbox = Text(description='Room Type:', disabled=True, style=style, layout=layout)
  roomSize_textbox = Text(description='Room Size:', disabled=True, style=style, layout=layout)
  normalPrice_textbox = FloatText(description='Normal Price:', disabled=True, style=style, layout=layout)
  memberPrice_textbox = FloatText(description='Member Price :', disabled=True, style=style, layout=layout)
  maxNoOfPet_textbox = Text(description='Maximum Number of Pets: ', disabled=True, style=style, layout=layout)
  suitableSpecies_textbox = Text(description='Suitable Species: ', disabled=True, style=style, layout=layout)
  amenities_textbox = Text(description='Amenities: ', disabled=True, style=style, layout=layout)
  description_textbox = Text(description='Room Description: ', disabled=True, style=style, layout=layout)

  def on_btnPick_clicked(b):
    c.execute("SELECT * FROM ROOM_TYPE WHERE RM_TYPENAME = '" + rm.value + "'")
    for row in c:
      roomType_textbox.value = row[-1]
      roomSize_textbox.value = row[1]
      normalPrice_textbox.value = row[2]
      memberPrice_textbox.value = row[3]
      maxNoOfPet_textbox.value = row[4]
      suitableSpecies_textbox.value = row[5]
      amenities_textbox.value = row[6]
      description_textbox.value = row[8]
  btnPick.on_click(on_btnPick_clicked)

  def on_btnGoToBook_button_clicked(b):
    bk_rmtype_dropdown.options = [row[0] for row in c.execute("SELECT RM_TYPENAME FROM ROOM_TYPE")]
    showPage("Make Booking")
  btnGoToBook.on_click(on_btnGoToBook_button_clicked)

  def on_btnC1Back_clicked(b):
    if login_status == "0":
      showPage("Introduction")
    if login_status == "1":
      showPage("Member Menu")
  btnC1Back.on_click(on_btnC1Back_clicked)

  roomSearching_row1HBox = widgets.HBox([roomType_textbox,roomSize_textbox])
  roomSearching_row2HBox = widgets.HBox([normalPrice_textbox, memberPrice_textbox])
  roomSearching_row3HBox = widgets.HBox([maxNoOfPet_textbox, suitableSpecies_textbox])
  roomSearching_row4HBox = widgets.HBox([amenities_textbox, description_textbox])
  roomSearching_row5HBox = widgets.HBox([btnGoToBook])
  roomSearching_row6HBox = widgets.HBox([btnC1Back])
  
  display(widgets.VBox([pickRoom, rm, btnPick, spacer1, showInfo, roomSearching_row1HBox, roomSearching_row2HBox,
                        roomSearching_row3HBox, roomSearching_row4HBox, roomSearching_row5HBox, roomSearching_row6HBox],layout=page_layout))
############# c1 - Room Searching Tab - End ##############

##########################################################
############# c2 - Make Booking Tab - Begin ############## -- Matthew, Allie, Jennie
##########################################################
with tb_all.output_to('Make Booking'):

  # create labels and boxes

  bk_msg = Label("")
  bk_spacer = Label("- - - - - - - - - -")
  bk_checkavail_msg = Label("")
  bk_checkmem_msg = Label("")
  bk_checkdisc_msg = Label("")

  bk_id_box = Text(disabled=True, placeholder='Booking ID')
  mem_id_box = Text(disabled=True, placeholder='Member ID')

  bk_rmtype_label = Label("Room type: ")
  bk_rmtype_options = [""]
  c.execute("SELECT RM_TYPENAME FROM ROOM_TYPE")
  for row in c:
    bk_rmtype_options.append(row[0])
  bk_rmtype_dropdown = widgets.Dropdown(options=bk_rmtype_options, value=None)

  bk_checkin_label = Label("Check in date: ")
  bk_checkin_datepicker = DatePicker(placeholder='Check in date: ', disabled=False)

  bk_checkout_label = Label("Check out date: ")
  bk_checkout_datepicker = DatePicker(placeholder='Check out date: ', disabled=False)

  bk_fname_label = Label("Your First Name: ")
  bk_fname_textbox = Text(placeholder='First Name', disabled=False)

  bk_lname_label = Label("Your Last Name: ")
  bk_lname_textbox = Text(placeholder='Last Name', disabled=False)

  bk_email_label = Label("Email: ")
  bk_email_textbox = Text(placeholder='Email Address', disabled=False)

  bk_phone_label = Label("Phone Number: ")
  bk_phone_textbox = Text(placeholder='Phone Number', disabled=False)

  bk_petname_label = Label("Pet Name: ")
  bk_petname_textbox = Text(placeholder='Pet Name', disabled=False)

  bk_species_label = Label("Pet Species: ")
  bk_species_textbox = Text(placeholder='Pet Species', disabled=True)

  bk_size_label = Label("Pet Size: ")
  bk_size_textbox = Text(placeholder='Pet Size', disabled=True)

  bk_request_label = Label("Special Request: ")
  bk_request_textbox = Text(placeholder='Special Request (Optional)', disabled=False)

  bk_pricepn_label = Label("Price per night: ")
  bk_pricepn_floatbox = FloatText(placeholder='Price per night', disabled=True)

  bk_numnight_label = Label("Number of nights: ")
  bk_numnight_intbox = IntText(placeholder='Number of nights', disabled=True)

  bk_totalprice_label = Label("Total price: ")
  bk_totalprice_floatbox = FloatText(placeholder='Total price', disabled=True)

  bk_discode_label = Label("Discount code: ")
  bk_discode_textbox = Text(placeholder='Discount code (Case-sensitive) (If any)', disabled=False)

  bk_mempriceoff_label = Label("Price off for Member: ")
  bk_mempriceoff_floatbox = FloatText(placeholder='Price off for Member', disabled=True)

  bk_discpriceoff_label = Label("Price off for Discount code: ")
  bk_discpriceoff_floatbox = FloatText(placeholder='Price off for Discount code', disabled=True)

  bk_actualpayment_label = Label("Price for this booking: ")
  bk_actualpayment_floatbox = FloatText(placeholder='Price for this booking', disabled=True)


  # create buttons

  bk_checkavail_button = widgets.Button(description="Check Availability")
  bk_reselect_button = widgets.Button(description="Reselect")
  bk_checkpriceoff_button = widgets.Button(description="Check for price off")
  bk_add_button = widgets.Button(description="Make Payment!", disabled=True)
  bk_clear_button = widgets.Button(description="Clear")
  bk_back_button = widgets.Button(description="Back")

  # define the event handlers for the buttons when they are being clicked
  def on_bk_checkavail_button_clicked(b):
    input_list1 = [bk_rmtype_dropdown.value, bk_checkin_datepicker.value, bk_checkout_datepicker.value]
    for i in input_list1:
      if i == None:
        bk_checkavail_msg.value = "Please select Room Type and Check in/Check out dates!"
        return
    if bk_checkin_datepicker.value > bk_checkout_datepicker.value:
      bk_checkavail_msg.value = "The check-in date should be before the check-out date. Please change it and try again."
      return
    elif bk_checkin_datepicker.value < date.today():
      bk_checkavail_msg.value = "The check-in date should not be before today. Please change it and try again."
      return

    import datetime
    abcdefghi="0"
    dateBooked=0
    for i in range((bk_checkout_datepicker.value-bk_checkin_datepicker.value).days):
        dateBooked=bk_checkin_datepicker.value + datetime.timedelta(days=i)
        c.execute("""SELECT * FROM (SELECT COUNT(*) TOTAL FROM ROOM WHERE ROOM_TYPE_RM_TYPEID=(SELECT RM_TYPEID FROM ROOM_TYPE WHERE RM_TYPENAME='{}')), (SELECT COUNT(*) BOOKED FROM BOOKING WHERE BK_RM_TYPE= (SELECT RM_TYPENAME FROM ROOM_TYPE WHERE RM_TYPENAME='{}') AND TO_DATE('{}','YYYY-MM-DD') BETWEEN BK_CHECKIN AND BK_CHECKOUT-1)""".format(bk_rmtype_dropdown.value, bk_rmtype_dropdown.value, dateBooked))
        b=[row[0]-row[1] for row in c][0]
        if b==0:
          abcdefghi="1"
    if abcdefghi=="1":
        bk_checkavail_msg.value = "No rooms available on chosen date."
        return

    else: 
        bk_checkavail_msg.value = "Room is available!"
        c.execute("select rm_suitablespecies from room_type where rm_typename='"+bk_rmtype_dropdown.value+"'")
        for row in c:
          bk_species_textbox.value = row[0]
        c.execute("select rm_suitablepetsize from room_type where rm_typename='"+bk_rmtype_dropdown.value+"'")
        for row in c:
          bk_size_textbox.value = row[0]
        c.execute("select rm_price from room_type where rm_typename='"+bk_rmtype_dropdown.value+"'")
        for row in c:
          bk_pricepn_floatbox.value = row[0]
        
        number_of_nights = bk_checkout_datepicker.value-bk_checkin_datepicker.value
        number_of_nights = number_of_nights.days
        bk_numnight_intbox.value=int(str(number_of_nights))

        bk_totalprice_floatbox.value = bk_pricepn_floatbox.value*bk_numnight_intbox.value
        bk_rmtype_dropdown.disabled=True
        bk_checkin_datepicker.disabled=True
        bk_checkout_datepicker.disabled=True
        bk_add_button.disabled=False
  bk_checkavail_button.on_click(on_bk_checkavail_button_clicked)


  def on_bk_reselect_button_clicked(b):
    bk_rmtype_dropdown.disabled=False
    bk_checkin_datepicker.disabled=False
    bk_checkout_datepicker.disabled=False
    bk_rmtype_dropdown.value=None
    bk_checkin_datepicker.value=None
    bk_checkout_datepicker.value=None
    bk_add_button.disabled=True
    bk_checkavail_msg.value = ""
  bk_reselect_button.on_click(on_bk_reselect_button_clicked)


  def on_bk_checkpriceoff_button_clicked(b):
    # check member part
    memtrue = 0
    c.execute("select count(mem_phone) from member where mem_phone ='"+bk_phone_textbox.value+"'")
    for row in c:
      memtrue = row[0]
    if memtrue == 1:
      memrates = 0.95
      bk_mempriceoff_floatbox.value = -round(((1-memrates)*bk_totalprice_floatbox.value),2)
      bk_checkmem_msg.value = "You are a Member! 5% off! "
      c.execute("select mem_id from member where mem_phone ='"+bk_phone_textbox.value+"'")
      for row in c:
        mem_id_record = row[0]
        mem_id_box.value = mem_id_record
    else:
      bk_mempriceoff_floatbox.value = 0
      bk_checkmem_msg.value = "You are not a member. Register for more discounts!"

    # check discount code part
    disctrue = 0
    c.execute("select count(dis_code) from discount where dis_code ='"+bk_discode_textbox.value+"'")
    for row in c:
      disctrue = row[0]
    if disctrue == 1:
      c.execute("select * from discount where dis_code ='"+bk_discode_textbox.value+"'")
      for row in c:
        codeee = row[0]
        rates = row[1]
        validfrom = row[2]
        validfrom = validfrom.date()
        validuntil = row[3]
        validuntil = validuntil.date()
        check0 = date.today()
      if validfrom <= check0 <= validuntil:
        bk_discpriceoff_floatbox.value = -round((rates*bk_totalprice_floatbox.value),2)
        bk_checkdisc_msg.value = "Valid Discount Code!"
      else:
        bk_discpriceoff_floatbox.value = 0
        bk_checkdisc_msg.value = "No Valid Discount Code"
    else:
      bk_discpriceoff_floatbox.value = 0
      bk_checkdisc_msg.value = "No Discount Code applied"

    # calculate actual price
    bk_actualpayment_floatbox.value = bk_totalprice_floatbox.value+bk_mempriceoff_floatbox.value+bk_discpriceoff_floatbox.value
  bk_checkpriceoff_button.on_click(on_bk_checkpriceoff_button_clicked)


  def on_bk_add_button_clicked(b):
    input_list2 = [bk_fname_textbox.value, bk_lname_textbox.value, bk_email_textbox.value, bk_phone_textbox.value, bk_petname_textbox.value]
    for i in input_list2:
        if i == None:
          bk_msg.value = "Please input all necessary information!"
          return
    
    c.execute("select max(bk_id) from booking")
    for row in c:
      max_bk_id = row[0]
      max_bk_id = int(max_bk_id)
      new_bk_id = max_bk_id+1
      new_bk_id = str(new_bk_id)
    bk_id_box.value = new_bk_id

    c.execute("INSERT INTO BOOKING (BK_ID, BK_RM_TYPE, BK_CHECKIN, BK_CHECKOUT, BK_FNAME, BK_LNAME, BK_EMAIL, BK_PHONE, BK_PET_NAME, BK_PET_SPECIES, BK_PET_SIZE, BK_SPECIALREQUEST, MEMBER_MEM_ID, DISCOUNT_DIS_CODE, BK_TOTALAMOUNT, BK_STATUS) VALUES ('{}', '{}', TO_DATE('{}','YYYY-MM-DD'), TO_DATE('{}','YYYY-MM-DD'), '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', {}, 'Waiting to be checked in')".format(bk_id_box.value, bk_rmtype_dropdown.value, bk_checkin_datepicker.value, bk_checkout_datepicker.value, bk_fname_textbox.value, bk_lname_textbox.value, bk_email_textbox.value, bk_phone_textbox.value, bk_petname_textbox.value, bk_species_textbox.value, bk_size_textbox.value, bk_request_textbox.value, mem_id_box.value, bk_discode_textbox.value, bk_actualpayment_floatbox.value))
    c.execute("commit")
    bk_msg.value = "Your booking ID is #"+bk_id_box.value+", payment and booking are successful! Thank you and see you soon!"
  bk_add_button.on_click(on_bk_add_button_clicked)


  def on_bk_clear_button_clicked(b):
    bk_fname_textbox.value = ""
    bk_lname_textbox.value = ""
    bk_email_textbox.value = ""
    bk_phone_textbox.value = ""
    bk_petname_textbox.value = ""
    bk_request_textbox.value = ""
    bk_discode_textbox.value = ""
    bk_msg.value = ""
  bk_clear_button.on_click(on_bk_clear_button_clicked)


  def on_bk_back_button_clicked(b):
    bk_rmtype_dropdown.value=None
    bk_checkin_datepicker.value=None
    bk_checkout_datepicker.value=None
    bk_fname_textbox.value = ""
    bk_lname_textbox.value = ""
    bk_email_textbox.value = ""
    bk_phone_textbox.value = ""
    bk_petname_textbox.value = ""
    bk_species_textbox.value = ""
    bk_size_textbox.value = ""
    bk_request_textbox.value = ""
    bk_pricepn_floatbox.value = 0
    bk_numnight_intbox.value = 0
    bk_totalprice_floatbox.value = 0
    bk_discode_textbox.value = ""
    bk_mempriceoff_floatbox.value = 0
    bk_discpriceoff_floatbox.value = 0
    bk_actualpayment_floatbox.value = 0  
    bk_msg.value = ""
    bk_checkavail_msg.value = ""
    bk_checkmem_msg.value = ""
    bk_checkdisc_msg.value = ""
    bk_add_button.disabled=True
    if login_status == "0":
      showPage("Introduction")
    if login_status == "1":
      showPage("Member Menu")
  bk_back_button.on_click(on_bk_back_button_clicked)


  # display the objects

  bk_vbox1 = widgets.VBox([bk_rmtype_label, bk_checkin_label])
  bk_vbox2 = widgets.VBox([bk_rmtype_dropdown, bk_checkin_datepicker])
  bk_hbox1 = widgets.HBox([bk_vbox1, bk_vbox2])
  bk_hbox2 = widgets.HBox([bk_checkout_label, bk_checkout_datepicker, bk_checkavail_button, bk_reselect_button])
  bk_part1 = widgets.VBox([bk_hbox1, bk_hbox2, bk_checkavail_msg, bk_spacer])

  bk_vbox3 = widgets.VBox([bk_fname_label, bk_lname_label, bk_email_label, bk_phone_label, bk_pricepn_label, bk_totalprice_label])
  bk_vbox4 = widgets.VBox([bk_fname_textbox, bk_lname_textbox, bk_email_textbox, bk_phone_textbox, bk_pricepn_floatbox, bk_totalprice_floatbox])
  bk_vbox5 = widgets.VBox([bk_petname_label, bk_species_label, bk_size_label, bk_request_label, bk_numnight_label, bk_discode_label])
  bk_vbox6 = widgets.VBox([bk_petname_textbox, bk_species_textbox, bk_size_textbox, bk_request_textbox, bk_numnight_intbox, bk_discode_textbox])
  bk_vbox7 = widgets.VBox([spacer,spacer,spacer,spacer,spacer,bk_checkpriceoff_button])
  bk_hbox3 = widgets.HBox([bk_vbox3, bk_vbox4, bk_vbox5, bk_vbox6, bk_vbox7])
  bk_hbox5 = widgets.HBox([bk_mempriceoff_label, bk_mempriceoff_floatbox, bk_checkmem_msg])
  bk_hbox6 = widgets.HBox([bk_discpriceoff_label, bk_discpriceoff_floatbox, bk_checkdisc_msg])
  bk_hbox7 = widgets.HBox([bk_actualpayment_label, bk_actualpayment_floatbox])
  bk_hbox8 = widgets.HBox([bk_msg])
  bk_hbox9 = widgets.HBox([bk_add_button, bk_clear_button])
  bk_hbox10 = widgets.HBox([bk_back_button])
  bk_part2 = widgets.VBox([bk_hbox3, bk_hbox5, bk_hbox6, bk_hbox7, bk_hbox8, bk_hbox9, bk_hbox10])

  display(widgets.VBox([bk_part1, bk_part2], layout=page_layout))
############# c2 - Make Booking Tab - End ################

##########################################################
############# c4 - Register Member - Begin ############### -- Gene
##########################################################
with tb_all.output_to('Register Member'):
  btnBack.on_click(on_btnBack_clicked)

  MEM_notEnoughData = Label("")
  MEM_msg = Label("")
  MEM_id_label = Label("Your Member ID: ")
  MEM_id_textbox = Text(placeholder='Your Member ID', disabled=True)
  mem_id_list = []
  c.execute('SELECT MEM_ID FROM MEMBER')
  for row in c:
    for i in row:
        mem_id_list.append(int(i))
    if mem_id_list == []:
      MEM_id_textbox.value = "1"
    else:
      MEM_id_textbox.value = str(max(mem_id_list)+1)

  #for row in c:
    #MEM_id_textbox.value = str(int(row[0])+1) 

  MEM_phone_label = Label("Phone: ")
  MEM_phone_textbox = BoundedIntText(placeholder='Phone: ', max=999999999,disabled=False)

  MEM_password_label = Label("Password: ")
  MEM_password_textbox = widgets.Password(placeholder='password', disabled=False) 

  MEM_sex_TYPE_label = Label("Gender: ")
  MEM_sex_TYPE_options = ["", "Female", "Male", "Others"]
  MEM_sex_TYPE = widgets.Dropdown(options=MEM_sex_TYPE_options, value=None)
  btnPick = widgets.Button(description="Pick")

  MEM_address_label = Label("Address: ")
  MEM_address_textbox =Text(placeholder='Address', disabled=False) 

  MEM_regday_label = Label("Register Date: ")
  MEM_regday_textbox = Text(placeholder='Member ID:', disabled=True)
  MEM_regday_textbox.value = str(date.today())

  MEM_add_button = widgets.Button(description="Register")
  MEM_clear_button = widgets.Button(description="Clear")

  def on_MEM_add_button_clicked(b):
    
    MEM_notEnoughData.value = ""
    
    #check data is valid or not
    input_list = [MEM_phone_textbox.value, MEM_password_textbox.value, MEM_sex_TYPE.value, MEM_address_textbox.value]
    for i in input_list:
      if i == "":
        MEM_notEnoughData.value = "Some of the data is missing. Please enter the missing text box and try again."
        return

    
    MEM_phone = str(MEM_phone_textbox.value)
    c.execute("SELECT COUNT(*) FROM MEMBER WHERE MEM_PHONE = '"+ MEM_phone +"' ")
    for abc in c:
      if abc[0] == 0:
        c.execute("INSERT INTO MEMBER (MEM_ID, MEM_PHONE, MEM_PASSWORD, MEM_GENDER, MEM_ADDRESS, MEM_REGISTRATIONDATE) VALUES ({},'{}','{}', '{}', '{}',TO_DATE('{}','YYYY-MM-DD'))".format(MEM_id_textbox.value, MEM_phone, MEM_password_textbox.value, MEM_sex_TYPE.value, MEM_address_textbox.value, MEM_regday_textbox.value))
        MEM_msg.value = "Register successfully! :)"
        
        c.execute("SELECT COUNT(*) FROM BOOKING WHERE BK_PHONE = '"+MEM_phone +"'")
        for defg in c:
          if defg[0] != 0:
            c.execute("UPDATE BOOKING SET MEMBER_MEM_ID = (SELECT MEM_ID FROM MEMBER WHERE MEM_ID = '"+  MEM_id_textbox.value +"') WHERE BK_PHONE = ('"+  MEM_phone +"') ")      
        c.execute("commit")
        MEM_add_button.disabled = True
        return
      else:
        MEM_msg.value = "phone registered"
  MEM_add_button.on_click(on_MEM_add_button_clicked)

  def on_MEM_clear_button_clicked(b):
    MEM_phone_textbox.value = 0
    MEM_password_textbox.value = ""
    MEM_sex_TYPE.value = "" 
    MEM_address_textbox.value = ""
    MEM_msg.value = ""
  MEM_clear_button.on_click(on_MEM_clear_button_clicked)

  MEM_labelVBox = widgets.VBox([MEM_id_label,MEM_phone_label, MEM_password_label,MEM_sex_TYPE_label,MEM_address_label,MEM_regday_label])
  MEM_textboxVBox = widgets.VBox([MEM_id_textbox,MEM_phone_textbox, MEM_password_textbox,MEM_sex_TYPE,MEM_address_textbox,MEM_regday_textbox])
  MEM_line1HBox = widgets.HBox([MEM_labelVBox, MEM_textboxVBox])
  MEM_line2HBox = widgets.HBox([MEM_msg])
  MEM_line3HBox = widgets.HBox([MEM_add_button, MEM_clear_button, btnBack])
  
  display(widgets.VBox([MEM_line1HBox, MEM_line2HBox, MEM_line3HBox, MEM_notEnoughData], layout=page_layout))
############## c4 - Register Member - End ################

##########################################################
################### Member Menu - Begin ################## -- Jennie
##########################################################
with tb_all.output_to('Member Menu'):
  # create welcoming labels
  memMenu_label1 = Label("Welcome Back!")

  # create 3 buttons
  btnM6 = widgets.Button(description="Redeem Rewards")
  btnMLogout = widgets.Button(description="Logout")

  # define the event handlers for the three buttons when they are being clicked
  btnC1.on_click(on_btnC1_button_clicked)
  btnC2.on_click(on_btnC2_button_clicked)

  def on_btnM6_clicked(b):
    showPage("Redeem Reward")
  btnM6.on_click(on_btnM6_clicked)

  def on_btnMLogout_clicked(b):
    phone_textbox.value = ""
    pwd_textbox.value = ""
    error_label.value = ""
    global login_id, login_status, mem_noofstar
    login_id = '0'
    login_status = '0'
    mem_noofstar=0
    showPage("Member Login")
  btnMLogout.on_click(on_btnMLogout_clicked)

  # display the objects
  memMenu_row1HBox = widgets.HBox([memMenu_label1])
  memMenu_row2HBox = widgets.HBox([spacer])
  memMenu_row3HBox = widgets.HBox([btnC1])
  memMenu_row4HBox = widgets.HBox([btnC2])
  memMenu_row5HBox = widgets.HBox([btnM6])
  memMenu_row6HBox = widgets.HBox([spacer])
  memMenu_row7HBox = widgets.HBox([btnMLogout])

  display(widgets.VBox([memMenu_row1HBox, memMenu_row2HBox, memMenu_row3HBox, memMenu_row4HBox, memMenu_row5HBox,
                        memMenu_row6HBox, memMenu_row7HBox], layout=page_layout))
################ Introduction Tab - End ##################

##########################################################
############# m6 - Redeem Reward Tab - Begin ############# -- Matthew
##########################################################
with tb_all.output_to('Redeem Reward'):
  btnMBack.on_click(on_btnMBack_clicked)

  pick_reward = Label("Select a reward")
  login_or_not = Label("")
  show_star = Label("")
  eligible = Label("")
  done_message = Label("")
  btnRedeem = widgets.Button(description="Redeem Reward")
  btnRedeem.disabled = True
  reward_options = [""]
  c.execute("SELECT RE_NAME FROM REWARD")
  for row in c:
    reward_options.append(row[0])
    reward = widgets.Dropdown(options=reward_options, value=None)
  
  showInfo = Label("Selected reward's information")

  style = {'description_width': '170px'}
  layout = {'width': '500px'}

  rewardName_textbox = Text(description='Reward Name:', disabled=True, style=style, layout=layout)
  rewardDescription_textbox = Text(description='Reward Description:', disabled=True, style=style, layout=layout)
  rewardStar_textbox = FloatText(description='Stars Required:', disabled=True, style=style, layout=layout)

  def on_btnPick_clicked(b):
    if login_status == "0":
      login_or_not.value = "Please login before you pick a reward."
    if login_status == "1":
      login_or_not.value = ""
      c.execute("SELECT * FROM REWARD WHERE RE_NAME = '" + reward.value + "'")
      for row in c:
        rewardName_textbox.value = row[1]
        rewardDescription_textbox.value = row[2]
        rewardStar_textbox.value = row[3]
      c.execute("SELECT MEM_STARS FROM MEMBER WHERE MEM_ID='"+login_id+"'")
      for row in c:
        star = row[0]
      if star >= rewardStar_textbox.value:
        eligible.value = 'You now have '+str(star)+' stars. Please press the "Redeem" button to redeem it.'
        btnRedeem.disabled = False
      else:
        eligible.value = 'You do not have enough stars. Please select another reward to redeem.' 
  btnPick.on_click(on_btnPick_clicked)

  def on_btnRedeem_clicked(b):
    c.execute('SELECT REDEEM_ID FROM REDEEM_RECORD')
    redeem_id_list = []
    for row in c:
      for i in row:
        redeem_id_list.append(int(i))
    if redeem_id_list == []:
      redeem_no = "1"
    else:
      redeem_no = max(redeem_id_list)+1
        
    c.execute("SELECT RE_ID FROM REWARD WHERE RE_NAME='"+rewardName_textbox.value+"'")
    for row in c:
      reward_id = row[0]
    c.execute("SELECT MEM_STARS FROM MEMBER WHERE MEM_ID='"+login_id+"'")
    for row in c:
      star = row[0]
    c.execute("INSERT INTO REDEEM_RECORD (REDEEM_ID, MEMBER_MEM_ID, REWARD_RE_ID, REDEEM_NUMBERREDEEMED, REDEEM_AMOUNTSPENT, REDEEM_DATE) VALUES('"+str(redeem_no)+"', '"+str(login_id)+"', '"+str(reward_id)+"', '1', '"+str(rewardStar_textbox.value)+"', SYSDATE)")
    c.execute("commit")

    c.execute("UPDATE MEMBER SET MEM_STARS=MEM_STARS-"+str(rewardStar_textbox.value)+" WHERE MEM_ID='"+str(login_id)+"'")
    c.execute("commit")
    done_message.value = "Your reward is sucessfully redeemed. You now have "+str(int(star-rewardStar_textbox.value))+" stars left."
    btnRedeem.disabled = True
  btnRedeem.on_click(on_btnRedeem_clicked)
    
  m6_HBox=widgets.HBox([btnRedeem, btnMBack])
  display(widgets.VBox([pick_reward, reward, btnPick, login_or_not, showInfo, rewardName_textbox, rewardDescription_textbox,
                        rewardStar_textbox, eligible, m6_HBox, done_message], layout=page_layout))
############# m6 - Redeem Reward Tab - End ###############

##########################################################
############### Member Login Tab - Begin ################# -- 
##########################################################
with tb_all.output_to('Member Login'):
  btnBack.on_click(on_btnBack_clicked)

  # label, textbox, button
  login_label = Label("Member Login")
  id_textbox = Text(description='ID:', disabled=False)
  phone_textbox = Text(description='Phone:', disabled=False)
  pwd_textbox = widgets.Password(description='Password:', disabled=False)
  error_label = Label("")
  login_button = widgets.Button(description="Login")

  # define the event handlers for buttons when being clicked
  def on_login_button_clicked(b):
    c.execute("SELECT MEM_ID FROM MEMBER WHERE MEM_PHONE = '" + phone_textbox.value + "'")
    for row in list(c):
      id_textbox.value = row[0]

    c.execute("SELECT COUNT(*) FROM MEMBER WHERE MEM_ID = '" + id_textbox.value + "' AND MEM_PASSWORD = '"+ pwd_textbox.value +"'")
    for row in list(c):
      if row[0]==0:
        error_label.value = "Wrong ID/Password, please try again."
      elif row[0]==1:
        c.execute("SELECT MEM_stars FROM MEMBER WHERE MEM_ID = '"+ id_textbox.value+"'")
        for row in list(c):
          global mem_noofstar
          global login_id, login_status
          login_id = id_textbox.value
          login_status = "1"
          if row[0] == None:
            mem_noofstar= 0
          else:
            mem_noofstar= int(row[0])
        showPage("Member Menu")
  login_button.on_click(on_login_button_clicked)

  # display
  line1HBox = widgets.HBox([login_label])
  line2HBox = widgets.HBox([phone_textbox])
  line3HBox = widgets.HBox([pwd_textbox])
  line4HBox = widgets.HBox([error_label])
  line5HBox = widgets.HBox([login_button, btnBack])

  display(widgets.VBox([line1HBox, line2HBox, line3HBox, line4HBox, line5HBox], layout=page_layout))
############### Member Login Tab - End ###################

##########################################################
################ Staff Login Tab - Begin ################# -- Jennie
##########################################################
with tb_all.output_to('Staff Login'):
  btnBack.on_click(on_btnBack_clicked)

  # create 2 labels
  stflogin_title = Label("Staff Login")
  stflogin_error = Label("")

  # create 2 textboxes
  stflogin_id_txt = Text(description='ID:', disabled=False)
  stflogin_pwd_txt = widgets.Password(description='Password:', disabled=False)

  # staff login button
  stflogin_btn = widgets.Button(description="Login")
  def on_stflogin_btn_clicked(b):
    c.execute("SELECT SF_TYPE FROM STAFF WHERE SF_ID='"+stflogin_id_txt.value+"' AND SF_PASSWORD='"+stflogin_pwd_txt.value+"'")
    stflogin_type = [row for row in c]
    if not stflogin_type:
      stflogin_error.value = "Wrong ID/Password, please try again."
    else:
      if stflogin_type[0][0] == "F":
        showPage("Staff Menu")
      elif stflogin_type[0][0] == "H":
        room_dropdown2.options = [row[0] for row in c.execute("SELECT RM_NO FROM ROOM WHERE RM_STATUS='To be cleaned' OR RM_STATUS='Cleaning'")]
        room_dropdown2.value = None
        showPage("Housekeeping Menu")
      elif stflogin_type[0][0] == "M":
        on_mgrdb_refresh_btn_clicked(b)
        showPage("Manager Dashboard")
  stflogin_btn.on_click(on_stflogin_btn_clicked)

  # display
  stflogin_HBox1 = widgets.HBox([stflogin_title])
  stflogin_HBox2 = widgets.HBox([stflogin_id_txt])
  stflogin_HBox3 = widgets.HBox([stflogin_pwd_txt])
  stflogin_HBox4 = widgets.HBox([stflogin_error])
  stflogin_HBox5 = widgets.HBox([stflogin_btn, btnBack])

  display(widgets.VBox([stflogin_HBox1, stflogin_HBox2, stflogin_HBox3, stflogin_HBox4, stflogin_HBox5], layout=page_layout))
################ Staff Login Tab - End ###################

##########################################################
################# Staff Menu Tab - Begin ################# -- Allie
##########################################################
with tb_all.output_to('Staff Menu'):
  btnSLogout.on_click(on_btnSLogout_clicked)

  # create 4 labels 
  stfmenu_title = Label("Staff Menu")
  stfmenu_labelbk = Label("BOOKING")
  stfmenu_labelins = Label("Insert...")
  stfmenu_labelupd = Label("Update...")
  stfmenu_labelrs = Label("Reset...")

  # create 5 buttons
  btnS1_1 = widgets.Button(description="New Room Type", style=dict(font_weight='bold'))
  btnS1_2 = widgets.Button(description="New Room Info", style=dict(font_weight='bold'))
  btnA1 = widgets.Button(description="Discount Code", style=dict(font_weight='bold'))
  btnS2 = widgets.Button(description="Room Info", style=dict(font_weight='bold'))
  btnS3_1 = widgets.Button(description="Room Availability", style=dict(font_weight='bold'))
  btnS4 = widgets.Button(description="View Details", style=dict(font_weight='bold'))
  btnS6 = widgets.Button(description="Check-in", style=dict(font_weight='bold'), layout = widgets.Layout(width='170px'))
  btnS7 = widgets.Button(description="Check-out", style=dict(font_weight='bold'), layout = widgets.Layout(width='170px'))
  btnS8 = widgets.Button(description="Member Password", style=dict(font_weight='bold'))
  btnSLogout = widgets.Button(description="Logout")

  # define the event handlers for the five buttons when they are being clicked
  def on_btnS1_1_clicked(b):
    showPage("Insert New Room Type")
  btnS1_1.on_click(on_btnS1_1_clicked)

  def on_btnS1_2_clicked(b):
    roomtypeid_dropdown.options = [row[0] for row in c.execute("SELECT RM_TYPENAME FROM ROOM_TYPE")]
    showPage("Insert New Room Info")
  btnS1_2.on_click(on_btnS1_2_clicked)

  def on_btnA1_clicked(b):
    showPage("Input Discount Codes")
  btnA1.on_click(on_btnA1_clicked)

  def on_btnS2_clicked(b):
    roomtypeid1_dropdown.options = [row[0] for row in c.execute("SELECT RM_TYPEID FROM ROOM_TYPE")]
    showPage("Update Room Type Info")
  btnS2.on_click(on_btnS2_clicked)

  def on_btnS3_1_clicked(b):
    room_dropdown1.options = [row[0] for row in c.execute("SELECT RM_NO FROM ROOM")]
    showPage("Update Room Availability")
  btnS3_1.on_click(on_btnS3_1_clicked)

  def on_btnS4_clicked(b):
    showPage("Search Booking")
  btnS4.on_click(on_btnS4_clicked)

  def on_btnS6_clicked(b):
    showPage("Checkin")
  btnS6.on_click(on_btnS6_clicked)

  def on_btnS7_clicked(b):
    showPage("Checkout")
  btnS7.on_click(on_btnS7_clicked)

  def on_btnS8_clicked(b):
    showPage("Reset Member Password")
  btnS8.on_click(on_btnS8_clicked)

  # display the objects
  stfmenu_row1HBox = widgets.HBox([stfmenu_title])
  stfmenu_row2HBox = widgets.HBox([spacer])
  stfmenu_row3HBox = widgets.HBox([stfmenu_labelbk])
  stfmenu_row4HBox = widgets.HBox([btnS4, btnS6, btnS7])
  stfmenu_row5HBox = widgets.HBox([spacer])
  stfmenu_row6HBox = widgets.HBox([stfmenu_labelins])
  stfmenu_row7HBox = widgets.HBox([btnS1_1, btnS1_2, btnA1])
  stfmenu_row8HBox = widgets.HBox([spacer])

  stfmenu_row9HBox = widgets.HBox([stfmenu_labelupd])
  stfmenu_row10HBox = widgets.HBox([btnS2, btnS3_1])
  stfmenu_VBox1 = widgets.VBox([stfmenu_row9HBox, stfmenu_row10HBox])
  stfmenu_VBox2 = widgets.VBox([stfmenu_labelrs, btnS8])

  stfmenu_row11HBox = widgets.HBox([stfmenu_VBox1, stfmenu_VBox2])
  stfmenu_row12HBox = widgets.HBox([spacer])
  stfmenu_row13HBox = widgets.HBox([btnSLogout])

  display(widgets.VBox([stfmenu_row1HBox, stfmenu_row2HBox, stfmenu_row3HBox, stfmenu_row4HBox, 
                        stfmenu_row5HBox, stfmenu_row6HBox, stfmenu_row7HBox, stfmenu_row8HBox,  
                        stfmenu_row11HBox, stfmenu_row12HBox, stfmenu_row13HBox], layout=page_layout))
################## Staff Menu Tab - End ##################

##########################################################
######### s1.1 - Insert New Room Type - Begin ############ -- Allie, Tom
##########################################################
with tb_all.output_to('Insert New Room Type'):
  btnFSBack.on_click(on_btnFSBack_clicked)

  # create labels and textboxes
  INS_msg = Label("")
  INS_intro = Label("Insert New Room Type")

  INS_rmtypeid_label = Label("Room Type ID: ")
  newTypeID = str(int([row[0] for row in c.execute("SELECT COUNT(RM_TYPEID) FROM ROOM_TYPE")][0])+1)
  INS_rmtypeid_textbox = Text(value = newTypeID, disabled = True)

  INS_rmtypename_label = Label("Room Type Name: ")
  INS_rmtypename_textbox = Text(placeholder = 'Standard (Cat), Premium (Dog)...', disabled = False)

  INS_rmtypeprice_label = Label("Room Price: ")
  INS_rmtypeprice_floatbox = FloatText(placeholder = 'Price', disabled = False)

  INS_rmtypesize_label = Label("Room Size (In m2): ")
  INS_rmtypesize_floatbox = FloatText(placeholder = 'In square feet (m2)', disabled = False)

  INS_rmmaxpet_label = Label("Max. No. of Pets: ")
  INS_rmmaxpet_intbox = IntText(placeholder = '1,2,3...', disabled = False)

  INS_rmsuitspecies_label = Label("Suitable Species: ")
  INS_rmsuitspecies_textbox = Text(placeholder = 'Cat, Dog...', disabled = False)

  INS_rmsuitpetsize_label = Label("Suitable Pet Size: ")
  INS_rmsuitpetsize_textbox = Text(placeholder = 'Small, Medium...', disabled = False)

  INS_rmamen_label = Label("Room Amenities: ")
  INS_rmamen_textbox = Text(placeholder = 'Bed, window...', disabled = False)

  INS_rmdesc_label = Label("Room Description: ")
  INS_rmdesc_textbox = Text(placeholder = 'Description', disabled = False)

  # create 3 buttons
  INS_add_button = widgets.Button(description = "Add Room Type", style=dict(font_weight='bold'))
  INS_clear_button = widgets.Button(description = "Clear")

  # define the event handlers for the 3 buttons when they are being clicked
  def on_INS_add_button_clicked(b):
    c.execute(
    """
      INSERT INTO ROOM_TYPE (RM_TYPEID,RM_TYPENAME, RM_PRICE, 
      RM_SIZE, RM_MAXNOOFPETS, RM_SUITABLESPECIES, RM_SUITABLEPETSIZE,
      RM_AMENITIES, RM_DESCRIPTION) 
      VALUES ('{}', '{}', {}, '{}', '{}', '{}', '{}', '{}', '{}') 
    """.format(INS_rmtypeid_textbox.value, INS_rmtypename_textbox.value, INS_rmtypeprice_floatbox.value,
               INS_rmtypesize_floatbox.value, INS_rmmaxpet_intbox.value, 
               INS_rmsuitspecies_textbox.value, INS_rmsuitpetsize_textbox.value, 
               INS_rmamen_textbox.value, INS_rmdesc_textbox.value
               )
    )
    INS_msg.value = "Inserted successfully! :)"
    c.execute("commit")
    INS_add_button.disabled = True
  INS_add_button.on_click(on_INS_add_button_clicked)

  def on_INS_clear_button_clicked(b):
    INS_rmtypeid_textbox.value = str(int([row[0] for row in c.execute("SELECT COUNT(RM_TYPEID) FROM ROOM_TYPE")][0])+1)
    INS_rmtypename_textbox.value = ""
    INS_rmtypeprice_floatbox.value = 0
    INS_rmtypesize_floatbox.value = 0
    INS_rmmaxpet_intbox.value = 0
    INS_rmsuitspecies_textbox.value = ""
    INS_rmsuitpetsize_textbox.value = ""
    INS_rmamen_textbox.value = ""
    INS_rmdesc_textbox.value = ""
    INS_add_button.disabled = False
    INS_msg.value = ""
  INS_clear_button.on_click(on_INS_clear_button_clicked)

  

  # display the objects
  INS_line0HBox = widgets.HBox([INS_intro])
  INS_line1HBox = widgets.HBox([INS_rmtypeid_label, INS_rmtypeid_textbox])
  INS_label1VBox = widgets.VBox([INS_rmtypename_label, INS_rmtypeprice_label, INS_rmtypesize_label, INS_rmamen_label, INS_rmdesc_label])
  INS_textbox1VBox = widgets.VBox([INS_rmtypename_textbox, INS_rmtypeprice_floatbox, INS_rmtypesize_floatbox, INS_rmamen_textbox, INS_rmdesc_textbox])
  INS_label2VBox = widgets.VBox([INS_rmmaxpet_label, INS_rmsuitspecies_label, INS_rmsuitpetsize_label])
  INS_textbox2VBox = widgets.VBox([INS_rmmaxpet_intbox, INS_rmsuitspecies_textbox, INS_rmsuitpetsize_textbox])
  INS_line2HBox = widgets.HBox([INS_label1VBox, INS_textbox1VBox, INS_label2VBox, INS_textbox2VBox])
  INS_line3HBox = widgets.HBox([INS_msg])
  INS_line4HBox = widgets.HBox([INS_add_button, INS_clear_button])
  INS_line5HBox = widgets.HBox([btnFSBack])

  display(widgets.VBox([INS_line0HBox, INS_line1HBox, INS_line2HBox, INS_line3HBox, INS_line4HBox, INS_line5HBox], layout=page_layout))
######### s1.1 - Insert Room Type Tab - End ##############

##########################################################
######### s1.2 - Insert New Room Info - Begin ############ -- Allie, Jovy
##########################################################
with tb_all.output_to('Insert New Room Info'):
  btnFSBack.on_click(on_btnFSBack_clicked)

  # create labels and textboxes
  INSs1_2_msg = Label("")
  INSs1_2_intro = Label("Insert New Room Info")

  INS_rmno_label = Label("Room No.: ")
  INS_rmno_textbox = BoundedIntText(value = None, min=0, max=99999, placeholder = 'Room No.', disabled = False)

  roomtypeid_label = Label("Room Type: ")
  roomtypeid_options=[]
  c.execute("SELECT RM_TYPENAME FROM ROOM_TYPE")
  for row in c:
    roomtypeid_options.append(row[0])
  roomtypeid_dropdown = widgets.Dropdown(options=roomtypeid_options, value=None)

  status_dropdown2_label = Label("Room Status: ")
  status_dropdown2 = widgets.Dropdown(options=["Available", "Unavailable", "Under Construction"], value=None)
  btns1_2Insert = widgets.Button(description="Insert", style=dict(font_weight='bold'))

  # define the event handlers for insert button when being clicked
  def on_btns1_2Insert_button_clicked(b):
    c.execute("SELECT RM_NO FROM ROOM")
    INSs1_2_rmno_list = [row[0] for row in c]
    if str(INS_rmno_textbox.value) in INSs1_2_rmno_list:
      INSs1_2_msg.value = "Duplicated room number, enter new room number."
    else:
      c.execute("SELECT RM_TYPEID id1 FROM ROOM_TYPE WHERE RM_TYPENAME = '{}'".format(roomtypeid_dropdown.value))
      for row in c:
        id1 = row[0]
      c.execute(
      """
      INSERT INTO ROOM (RM_NO, RM_STATUS, ROOM_TYPE_RM_TYPEID) 
      VALUES ('{}', '{}', '{}') 
      """.format(str(INS_rmno_textbox.value), status_dropdown2.value, id1
               )
      )
      INSs1_2_msg.value = "Inserted successfully!"
      c.execute("commit")
      INS_rmno_textbox.value = 0
      roomtypeid_dropdown.value = status_dropdown2.value = None
  btns1_2Insert.on_click(on_btns1_2Insert_button_clicked)

  # display
  s1_2_line1HBox = widgets.HBox([INSs1_2_intro])
  s1_2_vbox1 = widgets.VBox([INS_rmno_label, roomtypeid_label, status_dropdown2_label])
  s1_2_vbox2 = widgets.VBox([INS_rmno_textbox, roomtypeid_dropdown, status_dropdown2])
  s1_2_line2HBox = widgets.HBox([s1_2_vbox1, s1_2_vbox2])
  s1_2_line3HBox = widgets.HBox([INSs1_2_msg])
  s1_2_line4HBox = widgets.HBox([btns1_2Insert, btnFSBack])

  display(widgets.VBox([s1_2_line1HBox, s1_2_line2HBox, s1_2_line3HBox, s1_2_line4HBox], layout=page_layout))
######### s1.2 - Insert New Room Info Tab - End ##########

##########################################################
########## s2 - Update Room Type Info - Begin ############ -- Allie, Jennie
##########################################################
with tb_all.output_to('Update Room Type Info'):
  btnFSBack.on_click(on_btnFSBack_clicked)

  # create labels and textboxes
  UPD_intro = Label("Update Room Type Info")

  UPD_rmtypeid_label = Label("Room Type ID: ")
  roomtypeid1_options=[]
  c.execute("SELECT RM_TYPEID FROM ROOM_TYPE")
  for row in c:
    roomtypeid1_options.append(row[0])
  roomtypeid1_dropdown = widgets.Dropdown(options=roomtypeid1_options, value=None)

  UPD_msg = Label("")

  UPD_spacer = Label("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")

  UPD_rmtypename_label = Label("Room Type Name: ")
  UPD_rmtypename_textbox = Text(placeholder = 'Room Type Name', disabled = False)

  UPD_rmtypeprice_label = Label("Room Price: ")
  UPD_rmtypeprice_floatbox = FloatText(placeholder = 'Room Price', disabled = False)

  UPD_rmtypesize_label = Label("Room Size: ")
  UPD_rmtypesize_floatbox = FloatText(placeholder = 'In square meter (m2)', disabled = False)

  UPD_rmmaxpet_label = Label("Max. No. of Pets: ")
  UPD_rmmaxpet_intbox = IntText(placeholder = 'Max. No. of Pets', disabled = False)

  UPD_rmsuitspecies_label = Label("Suitable Species: ")
  UPD_rmsuitspecies_textbox = Text(placeholder = 'Suitable Species', disabled = False)

  UPD_rmsuitpetsize_label = Label("Suitable Pet Size: ")
  UPD_rmsuitpetsize_textbox = Text(placeholder = 'e.g. Small, Medium, Large', disabled = False)

  UPD_rmamen_label = Label("Room Amenities: ")
  UPD_rmamen_textbox = Text(placeholder = 'Room Amenities', disabled = False)

  UPD_rmdesc_label = Label("Room Description: ")
  UPD_rmdesc_textbox = Text(placeholder = 'Room Description', disabled = False)

  # create 4 buttons
  UPD_retrieve_button = widgets.Button(description = "Retrieve")
  UPD_update_button = widgets.Button(description = "Update", style=dict(font_weight='bold'))
  UPD_clear_button = widgets.Button(description = "Clear")

  # define the event handlers for the four buttons when they are being clicked
  def on_UPD_retrieve_button_clicked(b):
    c.execute("SELECT * FROM ROOM_TYPE WHERE RM_TYPEID = '" + roomtypeid1_dropdown.value + "'")
    for row in c:
      UPD_rmtypename_textbox.value = str(row[8])
      UPD_rmtypeprice_floatbox.value = str(row[2])
      UPD_rmtypesize_floatbox.value = str(row[1])
      UPD_rmmaxpet_intbox.value = str(row[3])
      UPD_rmsuitspecies_textbox.value = str(row[4])
      UPD_rmsuitpetsize_textbox.value = str(row[5])
      UPD_rmamen_textbox.value = str(row[6])
      UPD_rmdesc_textbox.value = str(row[7])
      UPD_msg.value = ""
  UPD_retrieve_button.on_click(on_UPD_retrieve_button_clicked)

  def on_UPD_clear_button_clicked(b):
    roomtypeid1_dropdown.value = None
    UPD_rmtypename_textbox.value = ""
    UPD_rmtypeprice_floatbox.value = 0
    UPD_rmtypesize_floatbox.value = 0
    UPD_rmmaxpet_intbox.value = 0
    UPD_rmsuitspecies_textbox.value = ""
    UPD_rmsuitpetsize_textbox.value = ""
    UPD_rmamen_textbox.value = ""
    UPD_rmdesc_textbox.value = ""
    UPD_msg.value = ""
  UPD_clear_button.on_click(on_UPD_clear_button_clicked)

  def on_UPD_update_button_clicked(b):
    c.execute(
    """
      UPDATE ROOM_TYPE 
      SET rm_typename = '{}',
      rm_price = {},
      rm_size = '{}',
      rm_maxnoofpets = '{}',
      rm_suitablespecies = '{}',
      rm_suitablepetsize = '{}',
      rm_amenities = '{}', 
      rm_description = '{}'
      WHERE rm_typeid = '{}'
    """.format(UPD_rmtypename_textbox.value, UPD_rmtypeprice_floatbox.value,
               UPD_rmtypesize_floatbox.value, UPD_rmmaxpet_intbox.value, 
               UPD_rmsuitspecies_textbox.value, UPD_rmsuitpetsize_textbox.value, 
               UPD_rmamen_textbox.value, UPD_rmdesc_textbox.value, 
               roomtypeid1_dropdown.value)
    )
    UPD_msg.value = "Updated successfully! :)"
    c.execute("commit")
  UPD_update_button.on_click(on_UPD_update_button_clicked)

  # display the objects
  UPD_line0HBox = widgets.HBox([UPD_intro])
  UPD_line1HBox = widgets.HBox([UPD_rmtypeid_label, roomtypeid1_dropdown, UPD_retrieve_button])
  UPD_line2HBox = widgets.HBox([UPD_spacer])
  UPD_line3HBox = widgets.HBox([UPD_rmtypename_label, UPD_rmtypename_textbox])
  UPD_label1VBox = widgets.VBox([UPD_rmtypeprice_label, UPD_rmtypesize_label, UPD_rmamen_label, UPD_rmdesc_label])
  UPD_textbox1VBox = widgets.VBox([UPD_rmtypeprice_floatbox, UPD_rmtypesize_floatbox, UPD_rmamen_textbox, UPD_rmdesc_textbox])
  UPD_label2VBox = widgets.VBox([UPD_rmmaxpet_label, UPD_rmsuitspecies_label, UPD_rmsuitpetsize_label])
  UPD_textbox2VBox = widgets.VBox([UPD_rmmaxpet_intbox, UPD_rmsuitspecies_textbox, UPD_rmsuitpetsize_textbox])
  UPD_line4HBox = widgets.HBox([UPD_label1VBox, UPD_textbox1VBox, UPD_label2VBox, UPD_textbox2VBox])
  UPD_line5HBox = widgets.HBox([UPD_msg])
  UPD_line6HBox = widgets.HBox([UPD_update_button, UPD_clear_button])
  UPD_line7HBox = widgets.HBox([btnFSBack])

  display(widgets.VBox([UPD_line0HBox, UPD_line1HBox, UPD_line2HBox, UPD_line3HBox, UPD_line4HBox, UPD_line5HBox, UPD_line6HBox, UPD_line7HBox], layout=page_layout))
########## s2 - Update Room Type Info - End ##############

##########################################################
###### s3.1 - Update Room Availability Tab - Begin ####### -- Allie, Jennie
##########################################################
with tb_all.output_to('Update Room Availability'):
  btnFSBack.on_click(on_btnFSBack_clicked)

  # label, dropdown, button
  updateStatus_label=Label("Update Room Availability Status")
  s3_label1 = Label("Room Number: ")
  s3_label2 = Label("Room Availability Status: ")
  s3_2_update_label1 = Label("")

  room_options = [row[0] for row in c.execute("SELECT RM_NO FROM ROOM")]
  room_dropdown1 = widgets.Dropdown(options=room_options, value=None)

  status1_dropdown = widgets.Dropdown(options=["Available", "Occupied", "To be cleaned"], value=None)
  btns31Update = widgets.Button(description="Update", style=dict(font_weight='bold'))
  
  # define the event handlers for update button when being clicked
  def on_btns31Update_button_clicked(b):
    c.execute("UPDATE ROOM SET RM_STATUS = '"+status1_dropdown.value+"' WHERE RM_NO = '"+room_dropdown1.value+"'")
    s3_2_update_label1.value = "Updated successfully! Room "+room_dropdown1.value+" is now "+status1_dropdown.value+" :)"
    c.execute("commit")
    room_dropdown1.value = status1_dropdown.value = None
  btns31Update.on_click(on_btns31Update_button_clicked)

  # display
  line1HBox = widgets.HBox([updateStatus_label])
  s3_1VBox1 = widgets.VBox([s3_label1, s3_label2])
  s3_1VBox2 = widgets.VBox([room_dropdown1, status1_dropdown])
  line2HBox = widgets.HBox([s3_1VBox1, s3_1VBox2])
  line3HBox = widgets.HBox([s3_2_update_label1])
  line4HBox = widgets.HBox([btns31Update, btnFSBack])

  display(widgets.VBox([line1HBox, line2HBox, line3HBox, line4HBox], layout=page_layout))
###### s3.1 - Update Room Availability Tab - End #########

##########################################################
########## s3.2 - Housekeeping Menu Tab - Begin ########## -- Jennie
##########################################################
with tb_all.output_to('Housekeeping Menu'):
  btnSLogout.on_click(on_btnSLogout_clicked)

  # label, dropdown, button
  updateStatus_label=Label("Update Room Cleaning Status")
  s3_label1 = Label("Room Number: ")
  s3_label3 = Label("Room Cleaning Status: ")

  room_dropdown2 = widgets.Dropdown(options=[], value=None)

  status2_dropdown = widgets.Dropdown(options=["Available", "Cleaning"], value=None)
  btns32Update = widgets.Button(description="Update", style=dict(font_weight='bold'))
  s3_2_update_label2 = Label("")

  # define the event handlers for update button when being clicked
  def on_btns32Update_button_clicked(b):
    c.execute("UPDATE ROOM SET RM_STATUS = '"+status2_dropdown.value+"' WHERE RM_NO = '"+room_dropdown2.value+"'")
    s3_2_update_label2.value = "Updated successfully! Room "+room_dropdown2.value+" is now "+status2_dropdown.value+" :)"
    c.execute("commit")
    room_dropdown2.options = [row[0] for row in c.execute("SELECT RM_NO FROM ROOM WHERE RM_STATUS='To be cleaned' OR RM_STATUS='Cleaning'")]
    room_dropdown2.value = status2_dropdown.value = None
  btns32Update.on_click(on_btns32Update_button_clicked)

  # display
  line1HBox = widgets.HBox([updateStatus_label])
  s3_2VBox1 = widgets.VBox([s3_label1, s3_label3])
  s3_2VBox2 = widgets.VBox([room_dropdown2, status2_dropdown])
  line2HBox = widgets.HBox([s3_2VBox1, s3_2VBox2])
  line3HBox = widgets.HBox([s3_2_update_label2])
  line4HBox = widgets.HBox([btns32Update, btnSLogout])

  display(widgets.VBox([line1HBox, line2HBox, line3HBox, line4HBox], layout=page_layout))
########## s3.2 - Housekeeping Menu Tab - End ############

##########################################################
############## S4 - Search Booking - Begin ############### -- Tom
##########################################################
with tb_all.output_to('Search Booking'):
  btnFSBack.on_click(on_btnFSBack_clicked)

  # label
  PetInfo_label = Label("Pet's Information:")
  BKInfo_label = Label("Booking's Information:")
  CusInfo_label = Label("Customer's Information:")
  
  # textbox
  Search_text = Text(description='Search Results:', placeholder='', disabled=True)
  BK_id_textbox2 = Text(description='Booking ID:', placeholder='Booking ID:', disabled=False) #0
  BK_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")
  BK_Fname_textbox2 = Text(placeholder='First Name:', description='First Name:', disabled=False) #6
  BK_Lname_textbox2 = Text(placeholder='Last Name:', description='Last Name:', disabled=False) #7
  BK_email_textbox2 = Text(placeholder='Email:', description='Email:', disabled=False) #5
  BK_Checkin_datepicker = DatePicker(description='Check in:', disabled=False) #2
  BK_Checkout_datepicker = DatePicker(description='Check out:', disabled=False) #3
  BK_Phone_textbox = Text(placeholder='Phone:', description='Phone:', disabled=False) #8
  BK_PetName_textbox = Text(placeholder='Pet Name:', description='Pet Name:', disabled=False) #9
  BK_PetSpecies_textbox = Text(placeholder='Pet Species:', description='Pet Species:', disabled=False) #10
  BK_PetSize_textbox = Text(placeholder='Pet Size:', description='Pet Size:', disabled=False) #11
  BK_RoomNo_textbox = Text(placeholder='Room No:', description='Room No:', disabled=False) #12
  BK_RoomType_textbox = Text(placeholder='Room Type:', description='Room Type:', disabled=False) #1
  BK_DisCode_textbox2 = Text(placeholder='Discount Code:', description='Discount Code:', disabled=False) #13
  BK_MemId_textbox = Text(placeholder='Member ID:', description='Member ID:', disabled=False) #14
  BK_SReq_textbox = Text(placeholder='Special Request:', description='Special Request:', disabled=False) #4
  BK_Total_textbox = Text(placeholder='Total Amount:', description='Total Amount:', disabled=False) #15
  BK_Status_textbox = Text(placeholder='Status:', description='Status:', disabled=False) #16

  # buttons
  BY_BK_id_btn = widgets.Button(description="Search by Booking ID")
  Search_btn = widgets.Button(description="Search")

  # define 
  def on_BY_BK_id_btn_clicked(b):
    c.execute("SELECT * FROM BOOKING WHERE BK_ID = '" + BK_id_textbox2.value + "'")
    BK_RoomType_textbox.value = ""
    BK_Checkin_datepicker.value = None
    BK_Checkout_datepicker.value = None
    BK_SReq_textbox.value = ""
    BK_email_textbox2.value = ""
    BK_Fname_textbox2.value = ""
    BK_Lname_textbox2.value = ""
    BK_Phone_textbox.value = ""
    BK_PetName_textbox.value = ""
    BK_PetSpecies_textbox.value = ""
    BK_PetSize_textbox.value = ""
    BK_RoomNo_textbox.value = ""
    BK_DisCode_textbox2.value = ""
    BK_MemId_textbox.value = ""
    BK_Total_textbox.value = ""
    BK_Status_textbox.value = ""

    for row in c:
      BK_RoomType_textbox.value = row[1]
      BK_Checkin_datepicker.value = row[2]
      BK_Checkout_datepicker.value = row[3]
      BK_SReq_textbox.value = str(row[4])
      BK_email_textbox2.value = str(row[5])
      BK_Fname_textbox2.value = str(row[6])
      BK_Lname_textbox2.value = str(row[7])
      BK_Phone_textbox.value = str(row[8])
      BK_PetName_textbox.value = str(row[9])
      BK_PetSpecies_textbox.value = str(row[10])
      BK_PetSize_textbox.value = str(row[11])
      BK_RoomNo_textbox.value = str(row[12])
      BK_DisCode_textbox2.value = str(row[13])
      BK_MemId_textbox.value = str(row[14])
      BK_Total_textbox.value = str(row[15])
      BK_Status_textbox.value = str(row[16])

  BY_BK_id_btn.on_click(on_BY_BK_id_btn_clicked)

  # Create dataframe for search results
  df = pd.DataFrame({"Booking_ID":[],"Room_Type":[],"Check_in":[],"First_Name":[],"Last_Name":[],"Phone":[],"Email":[]})
  id = []
  rt = []
  ci = []
  fn = []
  ln = []
  p = []
  e = []

  Search_output = widgets.Output()
  def on_Search_btn_clicked(b):
    df.drop(df.index, inplace=True)
    Search_output.clear_output()
    c.execute("SELECT * FROM BOOKING WHERE BK_FNAME LIKE '%" + BK_Fname_textbox2.value 
              + "%' AND BK_LNAME LIKE '%" + BK_Lname_textbox2.value 
              + "%' AND BK_EMAIL LIKE '%" + BK_email_textbox2.value
              + "%' AND BK_PHONE LIKE '%" + BK_Phone_textbox.value 
              + "%' ORDER BY BK_CHECKIN DESC")
    for row in c:
      id.append(row[0])
      rt.append(row[1])
      ci.append(row[2])
      fn.append(row[6])
      ln.append(row[7])
      p.append(row[8])
      e.append(row[9])
    df['Booking_ID'] = id
    df['Room_Type'] = rt
    df['Check_in'] = ci
    df['First_Name'] = fn
    df['Last_Name'] = ln
    df['Phone'] = p
    df['Email'] = e
    with Search_output:
      display(df)
  Search_btn.on_click(on_Search_btn_clicked)  


  # display 
  BK_line1HBox = widgets.HBox([BK_id_textbox2, BY_BK_id_btn])
  BK_spacer_0 = BK_spacer
  BK_line2HBox = widgets.HBox([BK_Fname_textbox2])
  BK_line3HBox = widgets.HBox([BK_Lname_textbox2])
  BK_line4HBox = widgets.HBox([BK_email_textbox2])
  BK_line5HBox = widgets.HBox([BK_Phone_textbox])  
  BK_spacer_1 = BK_spacer
  BK_line6HBox = widgets.HBox([BK_PetName_textbox, BK_PetSpecies_textbox, BK_PetSize_textbox])
  #BK_line7HBox = widgets.HBox([BK_PetWeight_textbox, BK_PetAge_textbox])
  BK_spacer_2 = BK_spacer
  BK_line8HBox = widgets.HBox([BK_RoomNo_textbox, BK_RoomType_textbox])
  BK_line8HBox_0 = widgets.HBox([BK_Checkin_datepicker, BK_Checkout_datepicker])
  BK_line8HBox_1 = widgets.HBox([BK_DisCode_textbox2, BK_MemId_textbox])
  BK_line8HBox_2 = widgets.HBox([BK_SReq_textbox])
  BK_spacer_3 = BK_spacer
  BK_line9HBox = widgets.HBox([BK_Total_textbox, BK_Status_textbox, spacer, btnFSBack])

  display(widgets.VBox([BK_line1HBox, BK_spacer_0, CusInfo_label, BK_line2HBox, BK_line3HBox, BK_line4HBox, 
          BK_line5HBox, BK_spacer_1, PetInfo_label, BK_line6HBox, 
          BK_spacer_2, BKInfo_label, BK_line8HBox, BK_line8HBox_0, BK_line8HBox_1, 
          BK_line8HBox_2, BK_spacer_3, BK_line9HBox],layout=page_layout))
############## S4 - Search Booking - End #################

##########################################################
################## s6 - Checkin - Begin ################## -- Allie
##########################################################
with tb_all.output_to('Checkin'):
  btnFSBack.on_click(on_btnFSBack_clicked)

  # create labels and textboxes
  CHK_intro = Label("Checkin")

  CHK_bkid_label = Label("Booking ID: ")
  CHK_bkid_textbox = Text(placeholder = 'Booking ID', disabled = False)
  
  CHK_msg = Label("")
  CHK_msg2 = Label("")

  CHK_spacer = Label("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")

  CHK_rmtype_label = Label("Room type: ")
  CHK_rmtype_textbox = Text(disabled = True)

  CHK_availrm_label = Label("Available rooms: ")

  CHK_status_label = Label("Checkin status: ")

  availrm_options = []
  CHK_availrm_dropdown = widgets.Dropdown(options=availrm_options, value=None)

    
  # create buttons
  CHK_retrieve_button = widgets.Button(description = "Retrieve")
  CHK_update_button = widgets.Button(description = "Check in", style=dict(font_weight='bold'))
  CHK_clear_button = widgets.Button(description = "Clear")


  # define the event handlers for the four buttons when they are being clicked
  def on_CHK_retrieve_button_clicked(b):
    c.execute("SELECT BK_RM_TYPE FROM BOOKING WHERE BK_ID = '"+CHK_bkid_textbox.value+"'")
    for row in c:
      CHK_rmtype_textbox.value = row[0]
      CHK_msg.value = ""
      availrm_options=[]
      c.execute("""SELECT RM_NO FROM ROOM WHERE RM_STATUS = 'Available' AND ROOM_TYPE_RM_TYPEID=
                (SELECT RM_TYPEID FROM ROOM_TYPE WHERE RM_TYPENAME='{}')""".format(CHK_rmtype_textbox.value))
      for row in c:
        availrm_options.append(row[0])
      CHK_availrm_dropdown.options = availrm_options
  CHK_retrieve_button.on_click(on_CHK_retrieve_button_clicked)


  def on_CHK_clear_button_clicked(b):
    CHK_bkid_textbox.value = ""
    CHK_msg.value = ""
    CHK_rmtype_textbox.value = ""
    CHK_msg2.value = ""
    CHK_availrm_dropdown.value = None
  CHK_clear_button.on_click(on_CHK_clear_button_clicked)

  def on_CHK_update_button_clicked(b):
    c.execute(
    """
      UPDATE BOOKING 
      SET BK_Status = 'Checked-in', ROOM_RM_NO='{}'
      WHERE BK_ID = '{}'
    """.format(CHK_availrm_dropdown.value, CHK_bkid_textbox.value)
    )
    c.execute("UPDATE ROOM SET RM_STATUS='Occupied' WHERE RM_NO='"+CHK_availrm_dropdown.value+"'")
    c.execute("commit")

    CHK_msg.value = "Checked in successfully! :)"

  CHK_update_button.on_click(on_CHK_update_button_clicked)



  # display the objects
  CHK_line0HBox = widgets.HBox([CHK_intro])
  CHK_line1HBox = widgets.HBox([CHK_bkid_label, CHK_bkid_textbox, CHK_retrieve_button])
  CHK_line2HBox = widgets.HBox([CHK_spacer])
  CHK_line3HBox = widgets.HBox([CHK_rmtype_label, CHK_rmtype_textbox])
  CHK_line4HBox = widgets.HBox([CHK_availrm_label, CHK_availrm_dropdown])

  CHK_line6HBox = widgets.HBox([CHK_msg])
  CHK_line7HBox = widgets.HBox([CHK_msg2])
  CHK_line8HBox = widgets.HBox([CHK_update_button, CHK_clear_button])
  CHK_line9HBox = widgets.HBox([btnFSBack])

  display(widgets.VBox([CHK_line0HBox, CHK_line1HBox, CHK_line2HBox, CHK_line3HBox, CHK_line4HBox,
                        CHK_line6HBox, CHK_line7HBox, CHK_line8HBox, CHK_line9HBox], layout=page_layout))
################### s6 - Checkin - End ###################

##########################################################
################## s7 - Checkout - Begin ################# -- Allie, Jovy
##########################################################
with tb_all.output_to('Checkout'):
  btnFSBack.on_click(on_btnFSBack_clicked)

  # create labels and boxes
  cko_label = Label("Checkout")
  cko_spacer = Label("- - - - - - - - - -")
  cko_msg = Label("")

  cko_rmid_label = Label("Room Number: ")
  cko_rmid_box = Text(placeholder='Room Number' ,disabled=False)

  cko_stars_label = Label("Stars earned for this booking: ")
  cko_stars_box = FloatText(disabled=True)

  cko_totalstars_label = Label("Total Stars: ")
  cko_totalstars_box = FloatText(disabled=True)

  # get value
  bk_id_box = Text()
  member_mem_id_box = Text()
  bk_totalamount_box = FloatText()
  bk_status_box = Text()
  rm_status_box = Text()

  # create buttons
  cko_retrieve_button = widgets.Button(description="Retrieve")
  cko_checkout_button = widgets.Button(description="Check out")
  cko_clear_button = widgets.Button(description="Clear")

  # define the event handlers for the buttons when they are being clicked
  def on_cko_retrieve_button_clicked(b):
    c.execute("select bk_id, member_mem_id, bk_totalamount, bk_status from booking where room_rm_no ='"+cko_rmid_box.value+"' and bk_status='Checked-in'")
    for row in c:
      bk_id_box.value = row[0]
      member_mem_id_box.value = str(row[1])
      bk_totalamount_box.value = row[2]
      bk_status_box.value = row[3]
    c.execute("select rm_status from room where rm_no ='"+cko_rmid_box.value+"'")
    for row in c:
      rm_status_box.value = row[0]
    # calculate stars
    getonestar = 250
    cko_stars_box.value = bk_totalamount_box.value/getonestar

    # old total stars
    if member_mem_id_box.value=="None":
      pass
    else:
      c.execute("select mem_stars from member where mem_id='"+member_mem_id_box.value+"'")
      for row in c:
        cko_totalstars_box.value = row[0]

    cko_rmid_box.disabled=True
  cko_retrieve_button.on_click(on_cko_retrieve_button_clicked)


  def on_cko_checkout_button_clicked(b):
    cko_totalstars_box.value = cko_totalstars_box.value+cko_stars_box.value
    newtotalstars = str(cko_totalstars_box.value)
    cko_stars_box.value = 0
    c.execute("update booking set bk_status='Checked-out' where bk_id='"+bk_id_box.value+"'")
    c.execute("update room set rm_status='To be cleaned' where rm_no='"+cko_rmid_box.value+"'")
    if member_mem_id_box.value=="None":
      pass
    else:
      c.execute("update member set mem_stars='"+newtotalstars+"' where mem_id='"+member_mem_id_box.value+"'")
    c.execute("commit")
    
    cko_rmid_box.disabled=False
    cko_msg.value = "Checked-out successfully and stars added!"
  cko_checkout_button.on_click(on_cko_checkout_button_clicked)


  def on_cko_clear_button_clicked(b):
    cko_rmid_box.value = ""
    cko_stars_box.value = 0
    cko_totalstars_box.value = 0
    cko_rmid_box.disabled=False
    cko_msg.value = ""
  cko_clear_button.on_click(on_cko_clear_button_clicked)


  # display the objects
  cko_hbox1 = widgets.HBox([cko_label])
  cko_hbox2 = widgets.HBox([cko_rmid_label, cko_rmid_box, cko_retrieve_button])
  cko_hbox3 = widgets.HBox([cko_spacer])
  cko_hbox4 = widgets.HBox([cko_stars_label, cko_stars_box])
  cko_hbox5 = widgets.HBox([cko_totalstars_label, cko_totalstars_box])
  cko_hbox6 = widgets.HBox([cko_msg])
  cko_hbox7 = widgets.HBox([cko_checkout_button, cko_clear_button])
  cko_hbox8 = widgets.HBox([btnFSBack])


  display(widgets.VBox([cko_hbox1, cko_hbox2, cko_hbox3, cko_hbox4, cko_hbox5, cko_hbox6, cko_hbox7, cko_hbox8], layout=page_layout))
################### s7 - Checkout - End ##################

##########################################################
########## s8 - Reset Member Password - Begin ############ -- Allie
##########################################################
with tb_all.output_to('Reset Member Password'):
  btnFSBack.on_click(on_btnFSBack_clicked)

  # create labels and textboxes
  RST_intro = Label("Reset Member Password")

  style2 = {'description_width': '150px'}
  layout2 = widgets.Layout(width='350px')

  RST_phone_textbox = Text(description = 'Member phone number: ', placeholder = 'Member phone number', disabled = False, style = style2, layout = layout2)
  
  RST_msg = Label("")

  RST_spacer = Label("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")

  RST_id_textbox = Text(description = 'Member ID: ', placeholder = 'Member ID', disabled = True, style = style2, layout = layout2)

  RST_pw_textbox = Text(description = 'Member password: ', placeholder = 'Member Password', disabled = False, style = style2, layout = layout2)

  # create 4 buttons
  RST_retrieve_button = widgets.Button(description = "Retrieve")
  RST_reset_button = widgets.Button(description = "Reset", style=dict(font_weight='bold'))
  RST_clear_button = widgets.Button(description = "Clear")

  # define the event handlers for the four buttons when they are being clicked
  def on_RST_retrieve_button_clicked(b):
    c.execute("SELECT MEM_ID, MEM_PASSWORD FROM MEMBER WHERE MEM_PHONE = '" + RST_phone_textbox.value + "'")
    for row in c:
      RST_id_textbox.value = row[0]
      RST_pw_textbox.value = row[1]
      RST_msg.value = ""
  RST_retrieve_button.on_click(on_RST_retrieve_button_clicked)

  def on_RST_clear_button_clicked(b):
    RST_phone_textbox.value = ""
    RST_id_textbox.value = ""
    RST_pw_textbox.value = ""
    RST_msg.value = ""
  RST_clear_button.on_click(on_RST_clear_button_clicked)

  def on_RST_reset_button_clicked(b):
    c.execute(
    """
      UPDATE MEMBER 
      SET mem_password = '{}'
      WHERE mem_phone = '{}'
    """.format(RST_pw_textbox.value, RST_phone_textbox.value)
    )
    RST_msg.value = "Reset successfully! :)"
    c.execute("commit")
  RST_reset_button.on_click(on_RST_reset_button_clicked)

  # display the objects
  RST_line0HBox = widgets.HBox([RST_intro])
  RST_line1HBox = widgets.HBox([RST_phone_textbox, RST_retrieve_button])
  RST_line2HBox = widgets.HBox([RST_spacer])
  RST_line3HBox = widgets.HBox([RST_id_textbox])
  RST_line4HBox = widgets.HBox([RST_pw_textbox])
  RST_line5HBox = widgets.HBox([RST_msg])
  RST_line6HBox = widgets.HBox([RST_reset_button, RST_clear_button])
  RST_line7HBox = widgets.HBox([btnFSBack])

  display(widgets.VBox([RST_line0HBox, RST_line1HBox, RST_line2HBox, RST_line3HBox, RST_line4HBox,
                        RST_line5HBox, RST_line6HBox, RST_line7HBox], layout=page_layout))
########### s8 - Update Member Password - End ############

##########################################################
########## s9 - Manager Dashboard Tab - Begin ############ -- Jennie
##########################################################
with tb_all.output_to('Manager Dashboard'):
  btnSLogout.on_click(on_btnSLogout_clicked)

  mgrdb_title = Label("Manager Dashboard")
  box_layout = Layout(border='1px solid white',
                      width='250px',
                      justify_content='space-between',
                      padding='7px',
                      margin='3px 0px 3px 0px')
  mgrdb_Out = widgets.Output()
  
  # dashboard refresh button
  mgrdb_refresh_btn = widgets.Button(description='Refresh')
  def on_mgrdb_refresh_btn_clicked(b):
    mgrdb_Out.clear_output()
  # r1 - total amount of daily/monthly income
    income_layout = Layout(flow='collumn',
                          border='1px solid white',
                          width='503px',
                          flex_flow='row',
                          justify_content='space-between',
                          padding='7px',
                          margin='3px 0px 3px 0px')
    
    # create 4 labels
    mgrdb_dayInc_lbl = Label("Daily income of", layout=Layout(width='110px'))
    mgrdb_dayInc_lbl2 = Label("", layout=Layout(width='100px'))
    mgrdb_mthInc_lbl = Label("Monthly income of", layout=Layout(width='110px'))
    mgrdb_mthInc_lbl2 = Label("", layout=Layout(width='100px'))

    # create 2 datepickers
    mgrdb_dayInc_date = widgets.DatePicker(disabled=False, layout=Layout(width='130px'))
    mgrdb_dayInc_date.value = date.today()
    mgrdb_mthInc_date = widgets.DatePicker(disabled=False, layout=Layout(width='130px'))
    mgrdb_mthInc_date.value = date.today()

    # daily income search button
    mgrdb_dayIncSearch_btn = widgets.Button(description="Search", layout=Layout(width='100px'), style=dict(font_weight='bold'))
    def on_mgrdb_dayIncSearch_btn_clicked(b):
      mgrdb_dayInc_chosenDate = str(mgrdb_dayInc_date.value.strftime('%d-%b-%y').upper())
      c.execute("SELECT SUM(BK_TOTALAMOUNT) FROM BOOKING WHERE BK_CHECKOUT = '"+mgrdb_dayInc_chosenDate+"'")
      for row in c:
        mgrdb_dayInc=row[0]
      mgrdb_dayInc_lbl2.value = "is $ "+str(mgrdb_dayInc)
    mgrdb_dayIncSearch_btn.on_click(on_mgrdb_dayIncSearch_btn_clicked)

    # monthly income search button
    mgrdb_mthIncSearch_btn = widgets.Button(description="Search", layout=Layout(width='100px'), style=dict(font_weight='bold'))
    def on_mgrdb_mthIncSearch_btn_clicked(b):
      mgrdb_mthInc_chosenDate = str(mgrdb_mthInc_date.value.strftime('01-%b-%y').upper())
      c.execute("SELECT SUM(BK_TOTALAMOUNT) FROM BOOKING WHERE BK_CHECKOUT BETWEEN '" + mgrdb_mthInc_chosenDate +
                "' AND LAST_DAY('" + mgrdb_mthInc_chosenDate + "')")
      for row in c:
        mgrdb_mthInc = row[0]
      mgrdb_mthInc_lbl2.value = "is $ "+str(mgrdb_mthInc)
    mgrdb_mthIncSearch_btn.on_click(on_mgrdb_mthIncSearch_btn_clicked)

    # r1 display boxes
    dis_r1_HBox1 = widgets.HBox([mgrdb_dayInc_lbl, mgrdb_dayInc_date, mgrdb_dayInc_lbl2, mgrdb_dayIncSearch_btn], layout=income_layout)
    dis_r1_HBox2 = widgets.HBox([mgrdb_mthInc_lbl, mgrdb_mthInc_date, mgrdb_mthInc_lbl2, mgrdb_mthIncSearch_btn], layout=income_layout)
  # end of r1

  # r2 - room bookings of today
    # define 3 layouts
    box_layout2 = Layout(border='1px solid white',
                      max_height='200px',
                        width='380px',
                        justify_content='space-between',
                        padding='5px',
                        margin='3px 0px 3px 0px')
    booking_layout = Layout(min_height='30px',justify_content='space-between',margin='0px 10px 5px 7px')
    booking_layout2 = Layout(border='0.5px solid white',
                              min_height='47px',
                              width='95%',
                              padding='7px',
                              margin='0px 0px 3px 7px')

    # create 6 labels
    mgrdb_tdyCheckin_lbl = Label("Check-in")
    mgrdb_tdyCheckin_lbl2 = Label("")
    mgrdb_tdyCheckout_lbl = Label("Check-out")
    mgrdb_tdyCheckout_lbl2 = Label("")
    mgrdb_tdyOvernight_lbl = Label("Overnight")
    mgrdb_tdyOvernight_lbl2 = Label("")

    # create 1 datepicker
    mgrdb_tdyBk_date = widgets.DatePicker(description='Bookings on', disabled=False, layout=Layout(width='230px'))
    mgrdb_tdyBk_date.value = date.today()

    # basic display
    r2_HBox1 = widgets.HBox([mgrdb_tdyCheckin_lbl, mgrdb_tdyCheckin_lbl2], layout=booking_layout)
    r2_HBox2 = widgets.HBox([mgrdb_tdyCheckout_lbl, mgrdb_tdyCheckout_lbl2], layout=booking_layout)
    r2_HBox3 = widgets.HBox([mgrdb_tdyOvernight_lbl, mgrdb_tdyOvernight_lbl2], layout=booking_layout)
    r2_Out = widgets.Output()

    # daily income search button
    mgrdb_tdyBkSearch_btn = widgets.Button(description="Search", style=dict(font_weight='bold'))
    def on_mgrdb_tdyBkSearch_btn_clicked(b):
      r2_Out.clear_output()
      mgrdb_tdyCheckin_list=[r2_HBox1]
      mgrdb_tdyCheckout_list=[r2_HBox2]
      mgrdb_tdyOvernight_list=[r2_HBox3]
      mgrdb_tdyBk_chosenDate = str(mgrdb_tdyBk_date.value.strftime('%d-%b-%y').upper())
      
      bookings_buttons = {}
      c.execute("""SELECT BK_ID, BK_FNAME, BK_LNAME, BK_RM_TYPE, BK_CHECKOUT-BK_CHECKIN Nights, BK_CHECKIN, BK_CHECKOUT
                FROM BOOKING WHERE '""" + mgrdb_tdyBk_chosenDate + "' BETWEEN BK_CHECKIN AND BK_CHECKOUT+1")  
      for row in c:   
        mgrdb_chosenBk_btn = widgets.Button(description="#"+str(row[0]), layout=Layout(width='60px', display='flex'),
                                            style=dict(font_weight='bold'))
        bookings_buttons[mgrdb_chosenBk_btn] = str(row[0])
        name_label = Label(str(row[1])+ " " + str(row[2]))
        roomType_label = Label(row[3])
        noOfNights_label = Label(str(row[4]) + " Nights")
        booking_HBox = widgets.HBox([mgrdb_chosenBk_btn, name_label, roomType_label, noOfNights_label],
                                    layout=Layout(justify_content='space-between'))
        booking = widgets.VBox([booking_HBox], layout=booking_layout2)
        if str(row[5]).split()[0]==mgrdb_tdyBk_date.value.strftime('%Y-%m-%d'):
          mgrdb_tdyCheckin_list.append(booking)
        elif str(row[6]).split()[0]==mgrdb_tdyBk_date.value.strftime('%Y-%m-%d'):
          mgrdb_tdyCheckout_list.append(booking)
        else:
          mgrdb_tdyOvernight_list.append(booking)
        
        # chosen booking details button
        def on_mgrdb_chosenBk_btn_clicked(b):
          r2_chosenBkId = bookings_buttons[b]          
          mgrVB_list1[0].value = "Booking #" + r2_chosenBkId
          
          # get booking details
          c.execute("""SELECT * FROM BOOKING, 
                    (SELECT CONCAT(CONCAT(BK_FNAME,' '),BK_LNAME) NAME, BK_CHECKOUT-BK_CHECKIN NIGHTS,
                    CONCAT(CONCAT(BK_CHECKIN, ' ~ '), BK_CHECKOUT) DATES FROM BOOKING WHERE BK_ID='{}')
                    WHERE BK_ID='{}'""".format(r2_chosenBkId, r2_chosenBkId))
          mgrVB_bkDetails = [row for row in c].pop()
          for i in range(15):
            mgrVB_list2[i+1].value = str(mgrVB_bkDetails[list(mgrVB_orderDict.values())[i]])
          showPage("View Booking")
        mgrdb_chosenBk_btn.on_click(on_mgrdb_chosenBk_btn_clicked)
      
      mgrdb_tdyCheckin_lbl2.value = str(len(mgrdb_tdyCheckin_list)-1)
      mgrdb_tdyCheckout_lbl2.value = str(len(mgrdb_tdyCheckout_list)-1)
      mgrdb_tdyOvernight_lbl2.value = str(len(mgrdb_tdyOvernight_list)-1)

      r2_VBox1 = widgets.VBox(mgrdb_tdyCheckin_list, layout=box_layout2)
      r2_VBox2 = widgets.VBox(mgrdb_tdyCheckout_list, layout=box_layout2)
      r2_VBox3 = widgets.VBox(mgrdb_tdyOvernight_list, layout=box_layout2)
      r2_VBox4 = widgets.VBox([r2_VBox1, r2_VBox2, r2_VBox3], layout=Layout(padding='3px'))
      with r2_Out:
        display(widgets.HBox([r2_VBox4]))
    mgrdb_tdyBkSearch_btn.on_click(on_mgrdb_tdyBkSearch_btn_clicked)

    # r2 display boxes
    r2_HBox = widgets.HBox([mgrdb_tdyBk_date, mgrdb_tdyBkSearch_btn])
    dis_r2_VBox1 = widgets.VBox([r2_HBox, r2_Out])
  # end of r2

  # r3 - 3 most popular room types & no of bookings
    mgrdb_3RT_lbl = Label("Top 3 Popular Room Type")
    mgrdb_3RT_list = [mgrdb_3RT_lbl]
    c.execute("SELECT * FROM (SELECT BK_RM_TYPE, COUNT(*) FROM BOOKING GROUP BY BK_RM_TYPE ORDER BY COUNT(*) DESC) WHERE ROWNUM<=3")
    for row in c:
      r3_roomType_lbl = Label(row[0])
      r3_noOfBookings_lbl = Label(str(row[1]))
      r3_HBox = widgets.HBox([r3_roomType_lbl, r3_noOfBookings_lbl], layout=Layout(justify_content='space-between'))
      mgrdb_3RT_list.append(r3_HBox)
    dis_r3_VBox1 = widgets.VBox(mgrdb_3RT_list, layout=box_layout)
  # end of r3

  # r4 - top 5 members & income
    mgrdb_5mem_lbl = Label("Top 5 members")
    mgrdb_5mem_list = [mgrdb_5mem_lbl]
    c.execute("""SELECT * FROM (SELECT MEM_ID, MEM_TOTALAMOUNTSPENT FROM MEMBER WHERE MEM_TOTALAMOUNTSPENT IS NOT NULL
              ORDER BY MEM_TOTALAMOUNTSPENT DESC) WHERE ROWNUM<=5""")
    for row in c:
      r3_mem_lbl = Label(row[0])
      r3_memInc_lbl = Label("$ "+str(row[1]))
      r4_HBox = widgets.HBox([r3_mem_lbl, r3_memInc_lbl], layout=Layout(justify_content='space-between'))
      mgrdb_5mem_list.append(r4_HBox)
    dis_r4_VBox1 = widgets.VBox(mgrdb_5mem_list, layout=box_layout)
  # end of r4

  # r5 - total income of each room types
    mgrdb_rtInc_lbl = Label("Room type income for #")
    mgrdb_rtInc_lbl2 = Label("")
    mgrdb_rtInc_list = [row[0] for row in c.execute("SELECT RM_TYPENAME FROM ROOM_TYPE ORDER BY RM_TYPENAME DESC")]
    mgrdb_rtInc_drop = widgets.Dropdown(options=mgrdb_rtInc_list, layout=Layout(width='160px'))

    # room type income search button
    mgrdb_rtIncSearch_btn = widgets.Button(description="Search", layout=Layout(width='65px'), style=dict(font_weight='bold'))
    def on_mgrdb_rtIncSearch_btn_clicked(b):
      c.execute("SELECT SUM(BK_TOTALAMOUNT) FROM BOOKING WHERE BK_RM_TYPE='" + mgrdb_rtInc_drop.value + "'")
      totalIncome = str([row[0] for row in c][0])
      mgrdb_rtInc_lbl2.value = "is $ "+totalIncome
    mgrdb_rtIncSearch_btn.on_click(on_mgrdb_rtIncSearch_btn_clicked)

    r5_HBox1 = widgets.HBox([mgrdb_rtInc_lbl, mgrdb_rtInc_lbl2])
    r5_HBox2 = widgets.HBox([mgrdb_rtInc_drop, mgrdb_rtIncSearch_btn])
    dis_r5_VBox1 = widgets.VBox([r5_HBox1, r5_HBox2], layout=box_layout)
  # end of r5

  # r6 - number of rooms in different status
    mgrdb_rmStatus_lbl = Label("Room Status")
    roomStatus_list = ["Available", "To be cleaned", "Cleaning", "Occupied", "Not in use", "Unavailable", "Under Construction"]
    mgrdb_rmStatus_list = [mgrdb_rmStatus_lbl]
    for s in roomStatus_list:
      mgrdb_rmStatus_lbl2 = Label(s)
      c.execute("SELECT COUNT(*) FROM ROOM WHERE RM_STATUS='" + s + "'")
      for row in c:
        mgrdb_rmStatus_lbl3 = Label(str(row[0]))
      r6_HBox = widgets.HBox([mgrdb_rmStatus_lbl2, mgrdb_rmStatus_lbl3], layout=Layout(justify_content='space-between'))
      mgrdb_rmStatus_list.append(r6_HBox)
    dis_r6_VBox1 = widgets.VBox(mgrdb_rmStatus_list, layout=box_layout)
  # end of r6

  #display
    mgrdb_VBox1 = widgets.VBox([dis_r1_HBox1, dis_r1_HBox2], layout=Layout(padding='3px'))
    mgrdb_VBox2 = widgets.VBox([dis_r5_VBox1, dis_r6_VBox1], layout=Layout(padding='3px'))
    mgrdb_VBox3 = widgets.VBox([dis_r3_VBox1, dis_r4_VBox1], layout=Layout(padding='3px'))
    mgrdb_HBox2 = widgets.HBox([mgrdb_VBox2, mgrdb_VBox3])
    mgrdb_VBox4 = widgets.VBox([mgrdb_VBox1, mgrdb_HBox2])
    mgrdb_VBox5 = widgets.VBox([dis_r2_VBox1])
    with mgrdb_Out:
      display(widgets.HBox([mgrdb_VBox4, mgrdb_VBox5]))
  mgrdb_refresh_btn.on_click(on_mgrdb_refresh_btn_clicked)

  mgrdb_HBox1 = widgets.HBox([mgrdb_title, mgrdb_refresh_btn, btnSLogout])
  display(widgets.VBox([mgrdb_HBox1, mgrdb_Out], layout=page_layout))
########### s9 - Manager Dashboard Tab - End #############

##########################################################
############# r2 - View Booking Tab - Begin ############## -- Jennie
##########################################################
with tb_all.output_to('View Booking'):
  mgrVB_orderDict = {"Name:":17, "Phone:":8, "Email:":5, "Room Type:":1, "Booked Date:":19, "Nights:":18,
                     "Special Request:":4, "Pet Name:":9, "Species:":10, "Size:":11, "Room Number:":12,
                     "Discount Code:":13, "Membership:":14, "Total Amount:":15, "Status:":16}
  mgrVB_list1 = [Label("")]
  mgrVB_list2 = [spacer]
  for i in mgrVB_orderDict.keys():
    mgrVB_list1.append(Label(i))
    mgrVB_list2.append(Text(value="", disabled=True))
  
  # back to manager dashboard button
  mgrVB_back_btn = widgets.Button(description="Back")
  def on_mgrVB_back_btn_clicked(b):
    showPage("Manager Dashboard")
  mgrVB_back_btn.on_click(on_mgrVB_back_btn_clicked)

  mgrVB_list1.append(spacer)
  mgrVB_list1.append(mgrVB_back_btn)

  # display
  mgrVB_VBox1 = widgets.VBox(mgrVB_list1)
  mgrVB_VBox2 = widgets.VBox(mgrVB_list2)
  display(widgets.HBox([mgrVB_VBox1, mgrVB_VBox2], layout=page_layout))
############# r2 - View Booking Tab - End ################

##########################################################
########## a1 - Input Discount Codes - Begin ############# -- Allie
##########################################################
with tb_all.output_to('Input Discount Codes'):
  btnFSBack.on_click(on_btnFSBack_clicked)

  # create labels and textboxes
  DIS_msg = Label("")
  DIS_intro = Label("Input Discount Codes")

  DIS_code_label = Label("Discount Code: ")
  DIS_code_textbox = Text(placeholder = 'Discount Code', disabled = False)

  DIS_rates_label = Label("Discount Rates: ")
  DIS_rates_floatbox = FloatText(disabled = False)
  DIS_note_label = Label("( 0 < discount rate < 1 )")

  DIS_startdate_label = Label("Discount Start Date: ")
  DIS_startdate_picker = widgets.DatePicker(disabled=False,layout=Layout(width='130px'))

  DIS_enddate_label = Label("Discount End Date: ")
  DIS_enddate_picker = widgets.DatePicker(disabled=False,layout=Layout(width='130px'))

  # create 3 buttons
  DIS_add_button = widgets.Button(description = "Input Discount", style=dict(font_weight='bold'))
  DIS_clear_button = widgets.Button(description = "Clear")

  # define the event handlers for the 3 buttons when they are being clicked
  def on_DIS_add_button_clicked(b):
    if DIS_code_textbox.value in [row[0] for row in c.execute("SELECT DIS_CODE FROM DISCOUNT")]:
      DIS_msg.value = "Invalid discount code."
    else:
      c.execute(
      """
        INSERT INTO DISCOUNT (DIS_CODE, DIS_RATES, DIS_STARTDATE, DIS_ENDDATE) 
        VALUES ('{}', {}, TO_DATE('{}','YYYY-MM-DD'), TO_DATE('{}','YYYY-MM-DD'))
      """.format(DIS_code_textbox.value, DIS_rates_floatbox.value, DIS_startdate_picker.value, DIS_enddate_picker.value)
      )
      DIS_msg.value = "Input successfully! :)"
      c.execute("commit")
      DIS_add_button.disabled = True       
  DIS_add_button.on_click(on_DIS_add_button_clicked)

  def on_DIS_clear_button_clicked(b):
    DIS_code_textbox.value = ""
    DIS_rates_floatbox.value = 0
    DIS_startdate_picker.value = date.today()
    DIS_enddate_picker.value = date.today()
    DIS_add_button.disabled = False
    DIS_msg.value = ""
  DIS_clear_button.on_click(on_DIS_clear_button_clicked)

  # display the objects
  DIS_line0HBox = widgets.HBox([DIS_intro])
  DIS_VBox1 = widgets.VBox([DIS_code_label, DIS_rates_label, DIS_startdate_label, DIS_enddate_label])
  DIS_rates_HBox = widgets.HBox([DIS_rates_floatbox, DIS_note_label])
  DIS_VBox2 = widgets.VBox([DIS_code_textbox, DIS_rates_HBox, DIS_startdate_picker, DIS_enddate_picker])
  DIS_line1HBox = widgets.HBox([DIS_VBox1, DIS_VBox2])
  DIS_line2HBox = widgets.HBox([DIS_msg])
  DIS_line3HBox = widgets.HBox([DIS_add_button, DIS_clear_button])
  DIS_line4HBox = widgets.HBox([btnFSBack])

  display(widgets.VBox([DIS_line0HBox, DIS_line1HBox, DIS_line2HBox, DIS_line3HBox, DIS_line4HBox],layout=page_layout))
########## a1 - Input Discount Codes Tab - End ###########

showPage("Introduction")

# integration by Jennie

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>